In [2]:
import pandas as pd

In [3]:
transactions_path = '/content/Transactions.csv'
products_path = '/content/Products.csv'
customers_path = '/content/Customers.csv'

In [5]:
# Load the data from the CSV files into pandas DataFrames
transactions_df = pd.read_csv(transactions_path) # Load transactions data
products_df = pd.read_csv(products_path) # Load products data
customers_df = pd.read_csv(customers_path) # Load customers data

In [6]:
transactions_info = transactions_df.info()
products_info = products_df.info()
customers_info = customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price            1000 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
<class 'pan

In [8]:
# Data cleaning and preprocessing: Convert dates to datetime for analysis
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])

In [9]:
# Check for duplicates and remove if any
transactions_df.drop_duplicates(inplace=True)
products_df.drop_duplicates(inplace=True)
customers_df.drop_duplicates(inplace=True)

In [10]:
# Merge datasets for comprehensive analysis
merged_df = transactions_df.merge(products_df, on='ProductID').merge(customers_df, on='CustomerID')

In [11]:
# Conduct basic analysis to understand sales patterns, top products, and regional trends
# 1. Top-selling products by quantity
top_products = merged_df.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).head(5)

In [16]:
# 2. Sales revenue by region
region_sales = merged_df.groupby('Region')['TotalValue'].sum().sort_values(ascending=False)


In [17]:
# 3. Monthly sales trends
merged_df['Month'] = merged_df['TransactionDate'].dt.to_period('M')
monthly_sales = merged_df.groupby('Month')['TotalValue'].sum()



In [18]:
 #4. Customer distribution by region
customer_region_distribution = customers_df['Region'].value_counts()

In [19]:
# 5. Product categories contributing to sales
category_sales = merged_df.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)

top_products, region_sales, monthly_sales, customer_region_distribution, category_sales

(ProductName
 ActiveWear Smartwatch    100
 SoundWave Headphones      97
 HomeSense Desk Lamp       81
 ActiveWear Rug            79
 SoundWave Cookbook        78
 Name: Quantity, dtype: int64,
 Region
 South America    219352.56
 Europe           166254.63
 North America    152313.40
 Asia             152074.97
 Name: TotalValue, dtype: float64,
 Month
 2023-12     3769.52
 2024-01    66376.39
 2024-02    51459.27
 2024-03    47828.73
 2024-04    57519.06
 2024-05    64527.74
 2024-06    48771.18
 2024-07    71366.39
 2024-08    63436.74
 2024-09    70603.75
 2024-10    47063.22
 2024-11    38224.37
 2024-12    59049.20
 Freq: M, Name: TotalValue, dtype: float64,
 Region
 South America    59
 Europe           50
 North America    46
 Asia             45
 Name: count, dtype: int64,
 Category
 Books          192147.47
 Electronics    180783.50
 Clothing       166170.66
 Home Decor     150893.93
 Name: TotalValue, dtype: float64)

In [20]:
# Customer Segmentation: Analyze total spend and purchase frequency by customer
customer_summary = merged_df.groupby('CustomerID').agg(
    TotalSpend=('TotalValue', 'sum'),
    PurchaseFrequency=('TransactionID', 'count'),
    AvgSpendPerPurchase=('TotalValue', 'mean')
).sort_values(by='TotalSpend', ascending=False)

In [21]:
repeat_customers = customer_summary[customer_summary['PurchaseFrequency'] > 1].shape[0]
one_time_customers = customer_summary[customer_summary['PurchaseFrequency'] == 1].shape[0]

In [22]:
# Calculate proportions
repeat_customer_percentage = (repeat_customers / len(customer_summary)) * 100
one_time_customer_percentage = (one_time_customers / len(customer_summary)) * 100

In [23]:
# Insights ready for next steps
customer_summary.head(), repeat_customer_percentage, one_time_customer_percentage

(            TotalSpend  PurchaseFrequency  AvgSpendPerPurchase
 CustomerID                                                    
 C0141         10673.87                 10          1067.387000
 C0054          8040.39                  8          1005.048750
 C0065          7663.70                 10           766.370000
 C0156          7634.45                 11           694.040909
 C0082          7572.91                  7          1081.844286,
 93.96984924623115,
 6.030150753768844)